<a href="https://colab.research.google.com/github/mjalalimanesh/statistical-learning-ISLR-python/blob/master/lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ISLR Chapter 5

# 5.3 Lab: Cross-Validation and the Bootstrap


In [1]:
# imports and setup
%matplotlib inline

import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd

pd.set_option('display.max_rows', 12)
plt.style.use('seaborn') # pretty matplotlib plots

In [10]:
# load data
auto = pd.read_csv('https://raw.githubusercontent.com/mjalalimanesh/statistical-learning-ISLR-python/master/datasets/Auto.csv', na_values='?')
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [11]:
auto.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower      float64
weight            int64
acceleration    float64
year              int64
origin            int64
name             object
dtype: object

In [12]:
auto.horsepower.unique()

array([130., 165., 150., 140., 198., 220., 215., 225., 190., 170., 160.,
        95.,  97.,  85.,  88.,  46.,  87.,  90., 113., 200., 210., 193.,
        nan, 100., 105., 175., 153., 180., 110.,  72.,  86.,  70.,  76.,
        65.,  69.,  60.,  80.,  54., 208., 155., 112.,  92., 145., 137.,
       158., 167.,  94., 107., 230.,  49.,  75.,  91., 122.,  67.,  83.,
        78.,  52.,  61.,  93., 148., 129.,  96.,  71.,  98., 115.,  53.,
        81.,  79., 120., 152., 102., 108.,  68.,  58., 149.,  89.,  63.,
        48.,  66., 139., 103., 125., 133., 138., 135., 142.,  77.,  62.,
       132.,  84.,  64.,  74., 116.,  82.])

In [13]:
auto.isnull().any()

mpg             False
cylinders       False
displacement    False
horsepower       True
weight          False
acceleration    False
year            False
origin          False
name            False
dtype: bool

In [14]:
auto.dropna(axis=0, inplace=True)

In [15]:
auto.cylinders = auto.cylinders.astype('category')

In [16]:
auto['horsepower_2'] = np.power(auto.horsepower, 2)
auto['horsepower_3'] = np.power(auto.horsepower, 3)
auto['horsepower_4'] = np.power(auto.horsepower, 4)
auto['horsepower_5'] = np.power(auto.horsepower, 5)

In [22]:
from sklearn.model_selection import train_test_split

X, y = auto.loc[:, ['horsepower', 'horsepower_2', 'horsepower_3']], auto.mpg

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=24)

## 5.3.1 The Validation Set Approach

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# ols model with intercept
lm1 = LinearRegression(fit_intercept=True)
lm2 = LinearRegression(fit_intercept=True)
lm3 = LinearRegression(fit_intercept=True)

lm1_fit = lm1.fit(X_train.loc[:, ['horsepower']], y_train)
lm2_fit = lm2.fit(X_train.loc[:, ['horsepower', 'horsepower_2']], y_train)
lm3_fit = lm3.fit(X_train.loc[:, ['horsepower', 'horsepower_2', 'horsepower_3']], y_train)

lm1_predict = lm1_fit.predict(X_test.loc[:, ['horsepower']])
lm2_predict = lm2_fit.predict(X_test.loc[:, ['horsepower', 'horsepower_2']])
lm3_predict = lm3_fit.predict(X_test.loc[:, ['horsepower', 'horsepower_2', 'horsepower_3']])

print('lm1 MSE:', mean_squared_error(y_test, lm1_predict))
print('lm2 MSE:', mean_squared_error(y_test, lm2_predict))
print('lm3 MSE:', mean_squared_error(y_test, lm3_predict))

lm1 MSE: 26.69873513822076
lm2 MSE: 20.945047941761047
lm3 MSE: 20.895213535496005


## 5.3.2 Leave-One-Out Cross-Validation

In [24]:
from sklearn.model_selection import LeaveOneOut

X, y = auto.loc[:, ['horsepower', 'horsepower_2', 'horsepower_3', 'horsepower_4', 'horsepower_5']], auto.mpg

loocv = LeaveOneOut()
loocv.get_n_splits(X)

392

In [25]:
loocv_mse = []
lm = LinearRegression(fit_intercept=True)

for train_index, test_index in loocv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    lm1_fit = lm.fit(X_train.loc[:, ['horsepower']], y_train)
    lm1_predict = lm1_fit.predict(X_test.loc[:, ['horsepower']])
    
    loocv_mse.append(mean_squared_error(y_test, lm1_predict))
    
np.array(loocv_mse).mean()

24.231513517929226

In [29]:
# using sklearn cross_validation_score
from sklearn.model_selection import cross_val_score

lm = LinearRegression(fit_intercept=True)

cval = cross_val_score(lm, 
                       auto.loc[:, ['horsepower']], 
                       auto.mpg, 
                       cv=len(auto), # k=n k-Fold -> LOOCV
                       n_jobs=-1, 
                       scoring='neg_mean_squared_error')
-cval.mean()

24.231513517929226

## 5.3.3 k-Fold Cross-Validation 

In [30]:
from sklearn.model_selection import KFold

X, y = auto.loc[:, ['horsepower', 'horsepower_2', 'horsepower_3', 'horsepower_4', 'horsepower_5']], auto.mpg

kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [35]:
# Loop for 5 degree polinomial linear regressions with k-Fold CV

kf_poly = {}

for i in range(1, 6):
    
    kf_mse = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        if i == 1:
            X_TRAIN = X_train.iloc[:,0:1].values.reshape(-1, 1)
            X_TEST = X_test.iloc[:,0:1].values.reshape(-1, 1)
        else:
            X_TRAIN = X_train.iloc[:,0:i]
            X_TEST = X_test.iloc[:,0:i]
        
        kf_mse.append(
            mean_squared_error(
                y_test, 
                LinearRegression(fit_intercept=True)
                .fit(
                    X_TRAIN, 
                    y_train
                )
                .predict(
                    X_TEST
                )
            )
        )
        
        kf_poly['lm' + str(i) + '_MSE'] = np.array(kf_mse).mean()

kf_poly

{'lm1_MSE': 24.199808197692477,
 'lm2_MSE': 19.228636614268016,
 'lm3_MSE': 19.26626534663179,
 'lm4_MSE': 19.35109227292693,
 'lm5_MSE': 19.02323339651184}

In [38]:
# using sklearn cross_validation_score
from sklearn.model_selection import cross_val_score

lm = LinearRegression(fit_intercept=True)

cval = cross_val_score(lm, 
                       X.loc[:, ['horsepower', 'horsepower_2']], 
                       y, 
                       cv=10, 
                       n_jobs=-1, 
                       scoring='neg_mean_squared_error')
-cval.mean()

21.235840055802225

## 5.3.4 The Bootstrap

In [40]:
portfolio = pd.read_csv('https://raw.githubusercontent.com/mjalalimanesh/statistical-learning-ISLR-python/master/datasets/Portfolio.csv', index_col=0)
portfolio.head()

,X,Y
1,-0.895251,-0.234924
2,-1.562454,-0.885176
3,-0.417090,0.271888
4,1.044356,-0.734198
5,-0.315568,0.841983


In [42]:
def alpha_fn(data, start_index, end_index):
    X = data['X'][start_index:end_index]
    Y = data['Y'][start_index:end_index]
    return ((np.var(Y) - np.cov(X, Y)[0][1]) / (np.var(X) + np.var(Y) - 2 * np.cov(X, Y)[0][1]))

In [43]:
alpha_fn(portfolio, 0, 100)

0.5766511516104118

In [44]:
from sklearn.utils import resample

portfolio_bs = resample(portfolio, replace=True, n_samples=100)

alpha_fn(portfolio_bs, 0, 100)

0.5632181593911421

In [45]:
# Bootstrap is removed from sklearn

bs_alpha = []

for i in range(0, 1000):
    bs_alpha.append(
        alpha_fn(resample(portfolio, replace=True, n_samples=100), 0, 100)
    )

bs_alpha = np.array(bs_alpha)

print('Bootstrapped alpha:', bs_alpha.mean())
print('SE:',  bs_alpha.std())

Bootstrapped alpha: 0.5791770366859285
SE: 0.09490562802044483


### Estimating accuracy of a Linear regression Model

In [46]:
def boot_fn(data, start_index, end_index):
    m = LinearRegression(fit_intercept=True).fit(
        data['horsepower'][start_index:end_index].values.reshape(-1, 1),
        data['mpg'][start_index:end_index]
    )
    
    return m.intercept_, m.coef_
    
boot_fn(auto, 0, 392)

(39.93586102117047, array([-0.15784473]))

In [47]:
boot_fn(resample(auto, replace=True, n_samples=392), 0, 392)

(38.95393827253029, array([-0.14646831]))

In [48]:
bs_boot = {'t1': [], 't2': []}

for i in range(0, 1000):
    bs_boot['t1'].append(
        boot_fn(resample(auto, replace=True, n_samples=392), 0, 392)[0]
    )
    bs_boot['t2'].append(
        boot_fn(resample(auto, replace=True, n_samples=392), 0, 392)[1][0]
    )

t1_es = np.array(bs_boot['t1']).mean()
t1_se = np.array(bs_boot['t1']).std()
t2_es = np.array(bs_boot['t2']).mean()
t2_se = np.array(bs_boot['t2']).std()

print('t1 bs estimate & se:', t1_es, t1_se)
print('t2 bs estimate & se:', t2_es, t2_se)

t1 bs estimate & se: 39.98682988953808 0.8471241597592658
t2 bs estimate & se: -0.15832415288371668 0.007601294489843631


In [50]:
import statsmodels.formula.api as sm

ols = sm.ols('mpg ~ horsepower', data=auto).fit()
print(ols.summary().tables[1])

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     39.9359      0.717     55.660      0.000      38.525      41.347
horsepower    -0.1578      0.006    -24.489      0.000      -0.171      -0.145


In [51]:
def boot_fn2(data, start_index, end_index):
    m = LinearRegression(fit_intercept=True).fit(
        data[['horsepower', 'horsepower_2']][start_index:end_index],
        data['mpg'][start_index:end_index]
    )
    
    return m.intercept_, m.coef_

In [52]:
bs_boot2 = {'t1': [], 't2': [], 't3': []}

for i in range(0, 1000):
    bs_boot2['t1'].append(
        boot_fn2(resample(auto, replace=True, n_samples=392), 0, 392)[0]
    )
    bs_boot2['t2'].append(
        boot_fn2(resample(auto, replace=True, n_samples=392), 0, 392)[1][0]
    )
    bs_boot2['t3'].append(
        boot_fn2(resample(auto, replace=True, n_samples=392), 0, 392)[1][1]
    )


t1_es = np.array(bs_boot2['t1']).mean()
t1_se = np.array(bs_boot2['t1']).std()
t2_es = np.array(bs_boot2['t2']).mean()
t2_se = np.array(bs_boot2['t2']).std()
t3_es = np.array(bs_boot2['t3']).mean()
t3_se = np.array(bs_boot2['t3']).std()

print('t1 bs estimate & se:', t1_es, t1_se)
print('t2 bs estimate & se:', t2_es, t2_se)
print('t3 bs estimate & se:', t3_es, t3_se)

t1 bs estimate & se: 56.91831002856803 2.1882644489496355
t2 bs estimate & se: -0.4679142773168464 0.03359322829124546
t3 bs estimate & se: 0.0012385339720369717 0.00011778213730320324


In [54]:
import statsmodels.formula.api as sm

ols2 = sm.ols('mpg ~ horsepower + horsepower_2', data=auto).fit()
print(ols2.summary().tables[1])

                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       56.9001      1.800     31.604      0.000      53.360      60.440
horsepower      -0.4662      0.031    -14.978      0.000      -0.527      -0.405
horsepower_2     0.0012      0.000     10.080      0.000       0.001       0.001
